In [1]:
(ql:quickload :cytoscape-clj)

To load "cytoscape-clj":
  Load 1 ASDF system:
    cytoscape-clj


(:CYTOSCAPE-CLJ)


; Loading "cytoscape-clj"



In [2]:
(defparameter *count* 0)
(defparameter +size+ 300)
(defparameter +margin+ 30)
(defparameter +columns+ 4)

(defparameter *cyto* (make-instance 'cytoscape:cytoscape-widget
                                    :layout (make-instance 'jupyter-widgets:layout :height "640px")
                                    :graph-style "node { content: data(name); } 
                                                  node.child { text-valign: center; text-halign: center; }
                                                  edge { curve-style: bezier; }"))

(defun element-id (id i)
  (format nil "~A-~A" id i))

(defun add-graphs (&rest layouts)
  (dolist (layout layouts)
  (let ((id (jupyter:make-uuid))
        (elements nil))
    (push (make-instance 'cytoscape:element      
                         :group "nodes" 
                         :data (list (cons "id" id)
                                     (cons "name" (string-downcase (class-name (class-of layout)))))
                         :classes (list id))
          elements)
    (dotimes (i 7)
        (push (make-instance 'cytoscape:element 
                             :group "nodes" 
                             :data (list (cons "id" (element-id id i))
                                         (cons "parent" id)
                                         (cons "name" (format nil "~A" i)))
                             :classes (list id "child"))
              elements))
    (dotimes (i 3)
        ; Edge from the corner to the next side
        (push (make-instance 'cytoscape:element 
                             :group "edges" 
                             :data (list (cons "parent" id)
                                         (cons "id" (jupyter:make-uuid))
                                         (cons "source" (element-id id (* 2 i)))
                                         (cons "target" (element-id id (1+ (* 2 i)))))
                             :classes (list id))
              elements)
        ; Edge from the corner to the previous side
        (push (make-instance 'cytoscape:element 
                             :group "edges" 
                             :data (list (cons "parent" id)
                                         (cons "id" (jupyter:make-uuid))
                                         (cons "source" (element-id id (* 2 i)))
                                         (cons "target" (element-id id (mod (1- (* 2 i)) 6))))
                             :classes (list id))
              elements)
        ; Edge from the corner to the middle
        (push (make-instance 'cytoscape:element 
                             :group "edges" 
                             :data (list (cons "parent" id)
                                         (cons "id" (jupyter:make-uuid))
                                         (cons "source" (element-id id (* 2 i)))
                                         (cons "target" (element-id id 6)))
                             :classes (list id))
              elements)
        ; Edge from the side to the next side
        (push (make-instance 'cytoscape:element 
                             :group "edges" 
                             :data (list (cons "parent" id)
                                         (cons "id" (jupyter:make-uuid))
                                         (cons "source" (element-id id (1+ (* 2 i))))
                                         (cons "target" (element-id id (mod (1- (* 2 i)) 6))))
                             :classes (list id))
              elements)
        ; Edge from the side to the middle
        (push (make-instance 'cytoscape:element 
                             :group "edges" 
                             :data (list (cons "parent" id)
                                         (cons "id" (jupyter:make-uuid))
                                         (cons "source" (element-id id (1+ (* 2 i))))
                                         (cons "target" (element-id id 6)))
                             :classes (list id))
              elements))
    (setf (cytoscape:selector layout) (format nil ".~A" id))
    (multiple-value-bind (row col) (floor *count* +columns+)
      (setf (cytoscape:bounding-box layout) (list (cons "x1" (* (+ +margin+ +size+) col))
                                                  (cons "y1" (* (+ +margin+ +size+) row))
                                                  (cons "w" +size+)
                                                  (cons "h" +size+))))
    (incf *count*)
    (setf (cytoscape:fit layout) nil)
    (cytoscape:on-layout-stop layout (lambda (l) (declare (ignore l)) (cytoscape:fit-elements *cyto*)))
    (setf (cytoscape:graph-layouts *cyto*) (append (cytoscape:graph-layouts *cyto*) (list layout)))
    (setf (cytoscape:elements *cyto*) (append (cytoscape:elements *cyto*) (reverse elements)))))
  (values))

*cyto*

*COUNT*

+SIZE+

+MARGIN+

+COLUMNS+

*CYTO*

ELEMENT-ID

ADD-GRAPHS

A Jupyter Widget

In [3]:
(add-graphs (make-instance 'cytoscape:breadth-first-layout)
            (make-instance 'cytoscape:random-layout)
            (make-instance 'cytoscape:grid-layout)
            (make-instance 'cytoscape:circle-layout)
            (make-instance 'cytoscape:concentric-layout)
            (make-instance 'cytoscape:cose-layout)
            (make-instance 'cytoscape:cola-layout)
            (make-instance 'cytoscape:dagre-layout))